##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Notice
Remember to enable GPU to make everything run faster (Runtime -> Change runtime type -> Hardware accelerator -> GPU).
Also, if you run into trouble, simply reset the entire environment and start from the beginning:
*   Edit -> Clear all outputs
*   Runtime -> Reset all runtimes

# Lab 04b: Dogs vs Cats Image Classification With Image Augmentation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/R/R04_C02_dogs_vs_cats_with_augmentation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/R/R04_C02_dogs_vs_cats_with_augmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this tutorial, we will discuss how to classify images into pictures of cats or pictures of dogs. We'll build an image classifier using a `keras_model_sequential()` model and load data using `keras` function `image_data_generator()`.

## Specific concepts that will be covered:
In the process, we will build practical experience and develop intuition around the following concepts

* Building _data input pipelines_ using the `image_data_generator()` class — How can we efficiently work with data on disk to interface with our model?
* _Overfitting_ - what is it, how to identify it, and how can we prevent it?
* _Data Augmentation_ and _Dropout_ - Key techniques to fight overfitting in computer vision tasks that we will incorporate into our data pipeline and image classifier model.

## We will follow the general machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build our model
4. Train our model
5. Test our model
6. Improve our model/Repeat the process

<hr>

**Before you begin**

Before running the code in this notebook, reset the runtime by going to **Runtime -> Reset all runtimes** in the menu above. If you have been working through several notebooks, this will help you avoid reaching Colab's memory limits.


# Install and load dependencies

First, you'll need to install and load R package Keras which will also install TensorFlow. We'll also install package `fs` which has useful functionality for working with our filesystem.

In [ ]:
install.packages(c("keras", "fs"))
library(keras)

# Data Loading

To build our image classifier, we begin by downloading the dataset. The dataset we are using is a filtered version of <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs. Cats</a> dataset from Kaggle (ultimately, this dataset is provided by Microsoft Research).

In previous Colabs, we've used datasets available through the `Keras` package, which is a very easy and convenient way to use datasets. In this Colab however, we will make use of the class `image_data_generator()` which will read data from disk. We therefore need to directly download *Dogs vs. Cats* from a URL and unzip it to the Colab filesystem.

In [ ]:
URL <- "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_dir <- get_file('cats_and_dogs_filtered.zip', origin = URL, extract = TRUE)

The dataset we have downloaded has the following directory structure.

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ...]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ...]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

We can list the directories with the following terminal command:

In [ ]:
zip_dir_base <- dirname(zip_dir)
fs::dir_tree(zip_dir_base, recurse = 2)

We'll now assign variables with the proper file path for the training and validation sets.

In [ ]:
base_dir <- fs::path(zip_dir_base, "cats_and_dogs_filtered")
train_dir <- fs::path(base_dir, "train")
validation_dir <- fs::path(base_dir, "validation")

train_cats_dir <- fs::path(train_dir, "cats")
train_dogs_dir <- fs::path(train_dir, "dogs")
validation_cats_dir <- fs::path(validation_dir, "cats")
validation_dogs_dir <- fs::path(validation_dir, "dogs")

### Understanding our data

Let's look at how many cats and dogs images we have in our training and validation directory

In [ ]:
num_cats_tr <- length(fs::dir_ls(train_cats_dir))
num_dogs_tr <- length(fs::dir_ls(train_dogs_dir))

num_cats_val <- length(fs::dir_ls(validation_cats_dir))
num_dogs_val <- length(fs::dir_ls(validation_dogs_dir))

total_train <- num_cats_tr + num_dogs_tr
total_val <- num_cats_val + num_dogs_val

In [ ]:
cat('total training cat images:', num_cats_tr, "\n")
cat('total training dog images:', num_dogs_tr, "\n")

cat('total validation cat images:', num_cats_val, "\n")
cat('total validation dog images:', num_dogs_val, "\n")
cat("--", "\n")
cat("Total training images:", total_train, "\n")
cat("Total validation images:", total_val, "\n")

# Setting Model Parameters

For convenience, we'll set up variables that will be used later while pre-processing our dataset and training our network.

In [ ]:
batch_size = 100  # Number of training examples to process before updating our models variables
img_shape  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels

# Data Augmentation

Overfitting often occurs when we have a small number of training examples. One way to fix this problem is to augment our dataset so that it has sufficient number and variety of training examples. Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples through random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This exposes the model to more aspects of the data, allowing it to generalize better.

In **keras** we can implement this using the same **`image_data_generator`** class we used before. We can simply pass different transformations we would want to our dataset as a form of arguments and it will take care of applying it to the dataset during our training process.

To start off, let's define a function that can display an image, so we can see the type of augmentation that has been performed. Then, we'll look at specific augmentations that we'll use during training.

In [ ]:
plot_rgb_image <- function(image_array){
  image_array %>%
  array_reshape(dim = c(dim(.)[1:3])) %>%
  as.raster(max = 1) %>%
  plot()
}

### Flipping the image horizontally

We can begin by randomly applying horizontal flip augmentation to our dataset and seeing how individual images will look after the transformation. This is achieved by passing `horizontal_flip = TRUE` as an argument to the `image_data_generator` class.

In [ ]:
train_image_generator <- image_data_generator(rescale = 1/255,
                                              horizontal_flip = TRUE)

train_data_gen <- flow_images_from_directory(directory = train_dir,
                                             generator = train_image_generator,
                                             target_size = c(img_shape, img_shape),
                                             class_mode = "binary",
                                             batch_size = batch_size)

To see the transformation in action, let's take one sample image from our training set and repeat it four times. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
n <- 4
augmented_training_images <- list(length = n)
for(i in 1:n){
 augmented_training_images[[i]] <- train_data_gen[1][[1]][1,,,]
}

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 4)

# Loop plotting over n images
layout(matrix(1:n, ncol = 4))

for(i in 1:n){
  augmented_training_images[[i]] %>%
  plot_rgb_image()
}


### Rotating the image

The rotation augmentation will randomly rotate the image up to a specified number of degrees. Here, we'll set it to 45.

In [ ]:
train_image_generator <- image_data_generator(rescale = 1/255,
                                              rotation_range = 45)

train_data_gen <- flow_images_from_directory(directory = train_dir,
                                             generator = train_image_generator,
                                             target_size = c(img_shape, img_shape),
                                             class_mode = "binary",
                                             batch_size = batch_size)

To see the transformation in action, let's once again take a sample image from our training set and repeat it. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
layout(matrix(1:n, ncol = 4))
for(i in 1:n){
 train_data_gen[1][[1]][1,,,] %>%
  plot_rgb_image()
}

### Applying Zoom

We can also apply Zoom augmentation to our dataset, zooming images up to 50% randomly.

In [ ]:
train_image_generator <- image_data_generator(rescale = 1/255,
                                              zoom_range = 0.5)

train_data_gen <- flow_images_from_directory(directory = train_dir,
                                             generator = train_image_generator,
                                             target_size = c(img_shape, img_shape),
                                             class_mode = "binary",
                                             batch_size = batch_size)


One more time, take a sample image from our training set and repeat it. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
layout(matrix(1:n, ncol = 4))
for(i in 1:n){
 train_data_gen[1][[1]][1,,,] %>%
  plot_rgb_image()
}

### Putting it all together

We can apply all these augmentations, and even others, with just one line of code, by passing the augmentations as arguments with proper values.

Here, we have applied rescale, rotation of 45 degrees, width shift, height shift, horizontal flip, and zoom augmentation to our training images.

In [ ]:
train_image_generator <- image_data_generator(rescale = 1/255,
                                              rotation_range = 45,
                                              width_shift_range = 0.2,
                                              height_shift_range = 0.2,
                                              shear_range = 0.2,
                                              zoom_range = 0.2,
                                              horizontal_flip = TRUE,
                                              fill_mode = 'nearest')

train_data_gen <- flow_images_from_directory(directory = train_dir,
                                             generator = train_image_generator,
                                             target_size = c(img_shape, img_shape),
                                             class_mode = "binary",
                                             batch_size = batch_size)

Let's visualize how a single image would look like four different times, when we pass these augmentations randomly to our dataset.

In [ ]:
layout(matrix(1:n, ncol = 4))
for(i in 1:n){
 train_data_gen[1][[1]][1,,,] %>%
  plot_rgb_image()
}

### Creating Validation Data generator

Generally, we only apply data augmentation to our training examples, since the original images should be representative of what our model needs to manage. So, in this case we are only rescaling our validation images and converting them into batches using `image_data_generator`.

In [ ]:
val_image_generator <- image_data_generator(rescale = 1/255)

val_data_gen <- flow_images_from_directory(directory = validation_dir,
                                             generator = val_image_generator,
                                             target_size = c(img_shape, img_shape),
                                             class_mode = "binary",
                                             batch_size = batch_size)

# Dropout

The Droput (`layer_dropout`) layer can be used to help prevent overfitting by randomly 'dropping out' the previous layer's outputs. The first parameter of the layer is the `rate`, a percentage of previous layer's outputs that will be dropped.

```r
# An example of how dropout is placed in a sequence of layers
layer_conv_2d(64, (3,3), activation='relu') %>%
layer_max_pooling_2d(2,2) %>%
layer_dropout(0.2), # Dropout layer added with a rate of 0.2
layer_conv_2d(64, (3,3), activation='relu') %>%
layer_max_pooling_2d(2,2)
```

# Exercise 4.4 Model Creation (15 mins) + break for training

Use what you've learned to create a CNN model, train it on the image generator we've created and visualise the results. You'll need to:

* **Define your model** with the the following layers. Add a `Dropout` layer with the rate of `0.5` after the last pooling layer:
    * 2D Convolution - `img_shape` x `img_shape` x 3 input shape, 32 filters, 3x3 kernel, ReLU activation
    * 2D Max pooling - 2x2 kernel
    * 2D Convolution - 64 filters, 3x3 kernel, ReLU activation
    * 2D Max pooling - 2x2 kernel
    * 2D Convolution - 128 filters, 3x3 kernel, ReLU activation
    * 2D Max pooling - 2x2 kernel
    * 2D Convolution - 128 filters, 3x3 kernel, ReLU activation
    * 2D Max pooling - 2x2 kernel
    * **Dropout - rate 0.5**
    * Flatten
    * Dense - 512 nodes, ReLU activation
    * Dense - 2 nodes, softmax activation 
* **Compile the model**
    * Use the `adam` optimizer, `sparse_categorial_crossentropy` loss and get the `accuracy` metric.
* Print out a summary of your model (optional)
* **Train the model** on the created image generators over 40 epochs
* **Print out and plot the output statistics** (`history`)

Check the documentation if you've forgotten any steps  [https://keras.rstudio.com/reference/index.htm](https://keras.rstudio.com/reference/index.htm)

In [ ]:
# TODO - Create, compile, and train the model. Then visualise the results.


## Exercise 4.4 Solution

The solution for the exercise can be found [here](https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/R/solutions/E.4.4.ipynb)

